In [40]:
!pip install tabula-py
!pip install PyPDF2

In [48]:
# PDF file
file = "/Users/wli/Documents/kg/1-s2.0-S0016703716301491-main.pdf"

In [43]:
import tabula
import PyPDF2

In [44]:
# document metadata
with open(file, "rb") as file:
    reader = PyPDF2.PdfReader(file)
    
    # Access document metadata and number of pages
    metadata = reader.metadata
    num_pages = len(reader.pages)
    
    # print("Metadata:", metadata)
    # print("Number of pages:", num_pages)

# save metadata to a text file
metadata_str = "\n".join([f"{key}: {value}" for key, value in metadata.items()])
with open("pdf_metadata.txt", "w") as meta_file:
    meta_file.write(metadata_str)
    print("Metadata saved to 'pdf_metadata.txt'")

Metadata saved to 'pdf_metadata.txt'


In [54]:
# Extract and save tables
for page in range(1, num_pages + 1):
    tables = tabula.read_pdf(file, pages=page, multiple_tables=True)
    if tables:
        for index, table in enumerate(tables):
            table.to_csv(f"table_{page}_{index + 1}.csv", index=False)
            print(f"Table {index+1} on page {page} saved as 'table_{page}_{index + 1}.csv'")
    else:
        print(f"Page {page}: No tables found.")

Table 1 on page 1 saved as 'table_1_1.csv'


In [61]:
import pandas as pd

data = pd.read_csv("table_1_1.csv")
# data

# separate the table
# table 1: Depth and Unit
table1 = data.iloc[0:2]

# Table 2: e-probe analyses to just before LA-ICP-MS analyses
table2 = data.iloc[2:14]

# Table 3: LA-ICP-MS analyses onwards (last rows)
table3 = data.iloc[14:]

In [74]:
# Find the index where "analyses" is in the first column
analyses_indices = data[data['Unnamed: 0'].str.contains('analyses', na=False)].index.tolist()
analyses_indices

[2, 15]

In [75]:
# Split the DataFrame at each 'analyses' row
tables = {}
start_idx = 0
for i, end_idx in enumerate(analyses_indices, start=1):
    tables[f"Table {i}"] = data.iloc[start_idx:end_idx]
    start_idx = end_idx
if start_idx < len(data):  # Include the last segment after the last 'analyses'
    tables[f"Table {len(analyses_indices) + 1}"] = data.iloc[start_idx:]

# Printing and saving each table
for name, table in tables.items():
    print(name, ":")
    print(table)
    print("\n")
    # Save each table to a CSV file
    table.to_csv(f"{name.replace(' ', '_').lower()}.csv", index=False)
    print(f"{name} saved to CSV.\n")

Table 1 :
      Unnamed: 0    SR684 SR686   SR689  Unnamed: 1   SR697   SR698   SR699  \
0  Depth (mbsla)  1766.5 1771.5  1779.4         NaN  1800.1  1802.5  1805.7   
1           Unit        248 248     248         NaN     254     255     256   

    SR700  Uncertainty BHVO-2g BHVO-2g GeoREM BCR-2g BCR-2g GeoREM  
0  1807.6          NaN     NaN            NaN    NaN           NaN  
1     256          NaN     NaN            NaN    NaN           NaN  


Table 1 saved to CSV.

Table 2 :
                                           Unnamed: 0      SR684 SR686  \
2   e-probe analyses (wt%) from Stolper et al. (2004)              NaN   
3                                                SiO2      48.19 48.83   
4                                                TiO2        2.63 2.62   
5                                               Al2O3      13.12 13.65   
6                                                FeOT      11.03 11.29   
7                                                 MnO        0.18 